# 한국어 혐오 발언 탐지

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 필요 라이브러리 설치

In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install torchmetrics
!pip install accelerate -U
!pip install scikit-learn
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-frfkvwma/kobert-tokenizer_3337928fb18340dcb738111c1d2e5dd9
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-frfkvwma/kobert-tokenizer_3337928fb18340dcb738111c1d2e5dd9
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


## 데이터셋 로드

- 학습, 검증, 테스트 데이터셋 준비
- 라벨 정보

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [2]:
from datasets import load_dataset

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [3]:
# 데이터 예제 출력

print(train)
print(validation)
print(test)
print(train['text'][0])
print(train['label'][0])

Dataset({
    features: ['text', 'label'],
    num_rows: 78977
})
Dataset({
    features: ['text', 'label'],
    num_rows: 8776
})
Dataset({
    features: ['text', 'label'],
    num_rows: 21939
})
"자한당틀딱들.. 악플질 고만해라."
[2, 4]


In [4]:
# 사전
print(train['text'][0:10])

# 리스트
print(train[0:10])


['"자한당틀딱들.. 악플질 고만해라."', '정치적으로 편향된 평론한은 분은 별로...', '적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...', '"안서는 아재들 풀발기 ㅋㄲㅋ"', '우와 ㅋ 능력자', '맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘', '주영훈 솔직히 호감임 잉꼬부부로 소문났잖아', '이게주간아이돌이랑머가달라...', '아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고!', '"문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ"']
{'text': ['"자한당틀딱들.. 악플질 고만해라."', '정치적으로 편향된 평론한은 분은 별로...', '적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...', '"안서는 아재들 풀발기 ㅋㄲㅋ"', '우와 ㅋ 능력자', '맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘', '주영훈 솔직히 호감임 잉꼬부부로 소문났잖아', '이게주간아이돌이랑머가달라...', '아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사람 짜증나게하냐 외국서 편히살려면 아닥하고 살아라 대한민국서 취미로 돈벌어가지말고 좀 끄지라고!', '"문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ"'], 'label': [[2, 4], [8], [2], [4], [8], [8], [8], [8], [3], [2, 3]]}


## 모델 및 토크나이저 로드

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

name_cards= ['monologg/koelectra-base-v3-discriminator', 'skt/kobert-base-v1', 'bert-base-multilingual-cased', 'snunlp/KR-BERT-char16424']
name_card = "snunlp/KR-BERT-char16424"
num_labels = 9
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if name_card == "skt/kobert-base-v1":
    from kobert_tokenizer import KoBERTTokenizer
    tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
else:
    tokenizer = AutoTokenizer.from_pretrained(name_card, do_lower_case=False, use_fast=False)
# print(tokenizer.pad_token_id)
model = AutoModelForSequenceClassification.from_pretrained(name_card, num_labels=num_labels, problem_type="multi_label_classification")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/104k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/397M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-BERT-char16424 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(16424, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
import torch.nn as nn
from transformers import AutoModel

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=1):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = AutoModel.from_pretrained(name_card)

        self.classifier = nn.Linear(self.model.config.hidden_size, num_labels)
        self.n_classifier = nn.Linear(self.model.config.hidden_size, num_labels)

        # self.intent_loss = nn.BCEWithLogitsLoss()
        self.intent_loss = FocalLoss()
        self.n_loss = nn.CrossEntropyLoss()


    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None, n_label=None):
        model_input= {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

        if token_type_ids is not None:
            model_input['token_type_ids'] = token_type_ids

        hidden = self.model(**model_input)

        # some output dont have pooler_output
        if hasattr(hidden, 'pooler_output'):
            cls = hidden.pooler_output
        else:
            cls = hidden.last_hidden_state[:, 0, :]

        hidden = hidden.last_hidden_state[:, 1:, :]

        intent_logit = self.classifier(hidden.mean(dim=1))
        n_logit = self.n_classifier(cls)

        intent_loss = self.intent_loss(intent_logit, labels)
        num_loss = self.n_loss(n_logit, n_label)

        loss = intent_loss + num_loss

        model_output = {
            'loss': loss,
            'intent_logit': intent_logit,
            'n_logit': n_logit,
        }

        return model_output


model = Model()
model.to(device)

Model(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(16424, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [7]:
# 토크나이저 예시

ids = tokenizer.encode(train['text'][0])
tokenized_words = tokenizer.convert_ids_to_tokens(ids)
model_input = tokenizer(train['text'][0])

print(tokenized_words)
print(ids)
print(model_input)

['[CLS]', '"', '자', '##한', '##당', '##틀', '##딱', '##들', '.', '.', '악', '##플', '##질', '고', '##만', '##해', '##라', '.', '"', '[SEP]']
[2, 11, 280, 20, 146, 1688, 8152, 283, 5, 5, 1043, 1370, 310, 37, 46, 39, 79, 5, 11, 3]
{'input_ids': [2, 11, 280, 20, 146, 1688, 8152, 283, 5, 5, 1043, 1370, 310, 37, 46, 39, 79, 5, 11, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
# 토크나이저 예시2

model_input = tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)
reverse = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
print(reverse)

{'input_ids': [2, 11, 280, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
['[CLS]', '"', '자', '[SEP]']


In [9]:
# 토크나이저 예시3

print(model_input)

another_name_card = 'roberta-base'
another_tokenizer = AutoTokenizer.from_pretrained(another_name_card, do_lower_case=False)
model_input = another_tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)

another_name_card = 'distilbert-base-uncased'
another_tokenizer = AutoTokenizer.from_pretrained(another_name_card, do_lower_case=False)
model_input = another_tokenizer(train['text'][0], max_length=4, truncation=True, padding="max_length")
print(model_input)


{'input_ids': [2, 11, 280, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [0, 113, 43998, 2], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': [101, 1000, 100, 102], 'attention_mask': [1, 1, 1, 1]}


In [10]:
from sklearn.preprocessing import MultiLabelBinarizer

def tokenize_function(examples):
    model_input = tokenizer(examples['text'], max_length=128, truncation=True, padding="max_length")

    mlb = MultiLabelBinarizer(classes=[0,1,2,3,4,5,6,7,8])
    one_hot_labels = mlb.fit_transform(examples['label'])

    model_input['label'] = one_hot_labels
    # model_input['n_label'] = []
    # for one_hot_label in one_hot_labels:
    #     if one_hot_label[-1] == 0:
    #         model_input['n_label'].append(sum(one_hot_label) - 1)
    #     else:
    #         model_input['n_label'].append(0)
    model_input['n_label'] = [sum(one_hot_label[:-1]) for one_hot_label in one_hot_labels] # 이거도 가능할 듯
    # model_input['n_label'] = [sum(one_hot_label) - 1 for one_hot_label in one_hot_labels]

    return model_input

In [11]:
print(train)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_valid = validation.map(tokenize_function, batched=True)

print(tokenized_train)

Dataset({
    features: ['text', 'label'],
    num_rows: 78977
})


Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'n_label'],
    num_rows: 78977
})


## 모델 학습

In [12]:
from transformers import TrainingArguments

BATCH_SIZE = 64 # 32
EPOCHS = 20

save_dir = '/root/korean_hate_speech_detection/model'

training_args = TrainingArguments(
    output_dir=save_dir,
    do_train=True,
    do_eval=True,
    # save_steps=999999999,
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    save_strategy="no",
    label_names=['labels', 'n_label'],
    seed=42,
)

In [13]:
import torch
from torchmetrics import Accuracy, F1Score, HammingDistance, AUROC, ExactMatch

def compute_metrics(eval_pred):
    threshold = 0.5

    # model_output = eval_pred.predictions
    # labels = eval_pred.label_ids
    intent_logits, n_logits = eval_pred.predictions
    intent_labels, n_labels = eval_pred.label_ids


    intent_logits = torch.Tensor(intent_logits)
    intent_labels = torch.Tensor(intent_labels).long()

    n_logits = torch.Tensor(n_logits)
    n_labels = torch.Tensor(n_labels).long()

    n_pred = torch.argmax(n_logits, dim=1)

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(intent_logits))

    preds = torch.zeros(size = probs.size())
    preds[probs >= threshold] = 1

    ...

    for b in range(preds.shape[0]):

        # hate: 1, no_hate: 1
        if preds[b][8] == 1:
            preds[b][:-1] = 0

        # hate: 0, no_hate: 0
        if preds[b][:-1].sum() == 0:
            preds[b][8] = 1

        # hate: 1, no_hate: 1
        if preds[b][:-1].sum() != 0:
            preds[b][8] = 0



    accuracy = Accuracy(task='multilabel', num_labels=9)
    f1_macro = F1Score(task="multilabel", num_labels=9, average='macro')
    f1_micro = F1Score(task="multilabel", num_labels=9, average='micro')
    f1_weight = F1Score(task="multilabel", num_labels=9, average='weighted')
    em = ExactMatch(task='multiclass', num_classes=2)
    auroc = AUROC(task='multilabel', num_labels=9, average='micro')
    hamming = HammingDistance(task="multiclass", num_classes=2)
    n_int_accuracy = Accuracy(task='multiclass', num_classes=9)

    f1s = F1Score(task='multilabel', num_labels=9, average=None)

    print("f1s")
    print(f1s(preds, intent_labels))


    metrics = {'accuracy': accuracy(preds, intent_labels),
               'f1_macro': f1_macro(preds, intent_labels),
               'f1_micro': f1_micro(preds, intent_labels),
               'f1_weighted': f1_weight(preds, intent_labels),
               'auroc': auroc(preds, intent_labels),
               'hamming_loss': hamming(preds, intent_labels),
               'em': em(preds, intent_labels),
               'n_int_accuracy': n_int_accuracy(n_pred, n_labels)}
    return metrics

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Hamming Loss,Em,N Int Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.679000,0.539966,0.955877,0.676458,0.821803,0.814926,0.893281,0.044123,0.778829,0.807885,17.699000,495.848000,7.797000
2,0.462600,0.497409,0.960967,0.705162,0.842856,0.838032,0.906199,0.039033,0.803783,0.824521,17.527200,500.708000,7.873000
3,0.357000,0.532821,0.962689,0.718005,0.850565,0.848101,0.912317,0.037311,0.811645,0.820761,17.530300,500.619000,7.872000
4,0.269100,0.588268,0.962296,0.735227,0.849733,0.849171,0.913517,0.037704,0.809708,0.822015,17.328600,506.447000,7.964000
5,0.200100,0.660823,0.963765,0.738455,0.854912,0.852096,0.914874,0.036235,0.818368,0.832270,17.518700,500.951000,7.877000
6,0.150200,0.729456,0.963486,0.734646,0.853693,0.850195,0.913938,0.036514,0.817229,0.830902,17.457800,502.699000,7.905000
7,0.116400,0.837791,0.963474,0.738159,0.854005,0.852296,0.914966,0.036526,0.817229,0.828851,17.559600,499.783000,7.859000
8,0.098300,0.892970,0.964600,0.755259,0.858473,0.855813,0.917423,0.035400,0.822242,0.832726,17.397400,504.445000,7.932000
9,0.080400,0.978396,0.963651,0.739123,0.855707,0.854600,0.918303,0.036349,0.817912,0.824635,17.379800,504.953000,7.940000
10,0.067900,1.001551,0.964626,0.741220,0.858273,0.855778,0.916574,0.035374,0.823382,0.832954,17.401200,504.334000,7.931000


f1s
tensor([0.6778, 0.7123, 0.7760, 0.8509, 0.7552, 0.6363, 0.0000, 0.7953, 0.8843])
f1s
tensor([0.7479, 0.7475, 0.8117, 0.8806, 0.7748, 0.6678, 0.0000, 0.8226, 0.8935])
f1s
tensor([0.7602, 0.7742, 0.8146, 0.8860, 0.7966, 0.7118, 0.0000, 0.8207, 0.8979])
f1s
tensor([0.7651, 0.7762, 0.8175, 0.8860, 0.8032, 0.7265, 0.1429, 0.8041, 0.8955])
f1s
tensor([0.7630, 0.7873, 0.8121, 0.8893, 0.7911, 0.7235, 0.1481, 0.8303, 0.9012])
f1s
tensor([0.7509, 0.7857, 0.8187, 0.8922, 0.7859, 0.7094, 0.1429, 0.8261, 0.9002])
f1s
tensor([0.7582, 0.7911, 0.8233, 0.8918, 0.8086, 0.7175, 0.1379, 0.8166, 0.8984])
f1s
tensor([0.7579, 0.7877, 0.8277, 0.8954, 0.8124, 0.7135, 0.2667, 0.8327, 0.9034])
f1s
tensor([0.7717, 0.7925, 0.8292, 0.8904, 0.8039, 0.7226, 0.1333, 0.8082, 0.9002])
f1s
tensor([0.7647, 0.7936, 0.8222, 0.8973, 0.8007, 0.7193, 0.1379, 0.8321, 0.9030])
f1s
tensor([0.7623, 0.7994, 0.8245, 0.8926, 0.8147, 0.7359, 0.1875, 0.8223, 0.8997])
f1s
tensor([0.7713, 0.7954, 0.8197, 0.8947, 0.8179, 0.7280, 0.322

TrainOutput(global_step=24700, training_loss=0.14049505604423493, metrics={'train_runtime': 7873.2607, 'train_samples_per_second': 200.621, 'train_steps_per_second': 3.137, 'total_flos': 0.0, 'train_loss': 0.14049505604423493, 'epoch': 20.0})

In [16]:
# trainer.save_model()
import os
torch.save(model.state_dict(), os.path.join(f"{save_dir}", "model.pt"))

## 예측

In [17]:
tokenized_test = test.map(tokenize_function, batched=True)

state_dict = torch.load(f"{save_dir}/model.pt")

model = Model()
model.load_state_dict(state_dict=state_dict)
model.to(device)

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

Model(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(16424, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [19]:
trainer.evaluate()

f1s
tensor([0.8039, 0.7772, 0.8294, 0.9066, 0.8245, 0.7423, 0.3059, 0.8287, 0.9004])


{'eval_loss': 1.439549207687378,
 'eval_accuracy': 0.9648216366767883,
 'eval_f1_macro': 0.7687655687332153,
 'eval_f1_micro': 0.8609854578971863,
 'eval_f1_weighted': 0.8600044846534729,
 'eval_auroc': 0.9197064638137817,
 'eval_hamming_loss': 0.03517836332321167,
 'eval_em': 0.8225534558296204,
 'eval_n_int_accuracy': 0.832490086555481,
 'eval_runtime': 42.7355,
 'eval_samples_per_second': 513.368,
 'eval_steps_per_second': 8.026}